In [1]:
# import the relevant libraries
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.stats import chi2_contingency
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import PolynomialFeatures
from tabulate import tabulate
from statsmodels.tools import add_constant
from datetime import datetime

In [2]:
# Load the data 
df = pd.read_csv('df1_wireframe.csv')
df. head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 9:45,58.9,13.29,1,credit_card,2,...,58.0,598.0,4.0,650.0,28.0,9.0,14.0,27277,volta redonda,SP
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 2:44,55.9,17.96,1,boleto,1,...,58.0,598.0,4.0,650.0,28.0,9.0,14.0,27277,volta redonda,SP
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56,64.9,18.33,1,credit_card,2,...,58.0,598.0,4.0,650.0,28.0,9.0,14.0,27277,volta redonda,SP
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55,58.9,16.17,1,credit_card,3,...,58.0,598.0,4.0,650.0,28.0,9.0,14.0,27277,volta redonda,SP
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05,58.9,13.29,1,credit_card,4,...,58.0,598.0,4.0,650.0,28.0,9.0,14.0,27277,volta redonda,SP


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117329 entries, 0 to 117328
Data columns (total 38 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       117329 non-null  object 
 1   order_item_id                  117329 non-null  int64  
 2   product_id                     117329 non-null  object 
 3   seller_id                      117329 non-null  object 
 4   shipping_limit_date            117329 non-null  object 
 5   price                          117329 non-null  float64
 6   freight_value                  117329 non-null  float64
 7   payment_sequential             117329 non-null  int64  
 8   payment_type                   117329 non-null  object 
 9   payment_installments           117329 non-null  int64  
 10  payment_value                  117329 non-null  float64
 11  customer_id                    117329 non-null  object 
 12  order_status                  

In [4]:
df.describe()

,order_item_id,price,freight_value,payment_sequential,payment_installments,payment_value,review_score,postal_code,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix
count,117329.000000,117329.000000,117329.000000,117329.000000,117329.000000,117329.000000,117329.000000,117329.000000,115634.000000,115634.000000,115634.000000,117309.000000,117309.000000,117309.000000,117309.000000,117329.000000
mean,1.194121,120.524349,20.027514,1.094452,2.940151,172.062565,4.031467,35060.118112,48.768018,785.802861,2.205528,2110.763062,30.254456,16.612476,23.071452,24450.781955
std,0.684225,182.944843,15.828077,0.731174,2.775370,265.388194,1.387927,29849.496175,10.033831,652.382965,1.717783,3785.128931,16.177519,13.452625,11.745875,27582.364358
min,1.000000,0.850000,0.000000,1.000000,0.000000,0.000000,1.000000,1003.000000,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000,1001.000000
25%,1.000000,39.900000,13.080000,1.000000,1.000000,60.750000,4.000000,11250.000000,42.000000,346.000000,1.000000,300.000000,18.000000,8.000000,15.000000,6429.000000
50%,1.000000,74.900000,16.280000,1.000000,2.000000,108.100000,5.000000,24240.000000,52.000000,600.000000,1.000000,700.000000,25.000000,13.000000,20.000000,13660.000000
75%,1.000000,134.900000,21.180000,1.000000,4.000000,189.060000,5.000000,58770.000000,57.000000,983.000000,3.000000,1800.000000,38.000000,20.000000,30.000000,28035.000000
max,21.000000,6735.000000,409.680000,29.000000,24.000000,13664.080000,5.000000,99990.000000,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000,99730.000000


In [5]:
df.isnull().sum().sum()

181668

In [6]:
missing_columns=df.columns[df.isnull().any()].tolist()
print(missing_columns)

['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'review_comment_title', 'review_comment_message', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']


In [7]:
df[missing_columns].isnull().sum()

order_approved_at                    15
order_delivered_carrier_date       1235
order_delivered_customer_date      2471
review_comment_title             103437
review_comment_message            67650
product_category_name              1695
product_name_lenght                1695
product_description_lenght         1695
product_photos_qty                 1695
product_weight_g                     20
product_length_cm                    20
product_height_cm                    20
product_width_cm                     20
dtype: int64

In [8]:

# columns to convert
columns_to_convert = ["order_approved_at", "order_delivered_customer_date", "shipping_limit_date", "order_purchase_timestamp", "order_delivered_carrier_date", "order_estimated_delivery_date", "review_answer_timestamp"]

# Convert the specified columns to datetime
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_datetime)

# Verify the updated column data types
print(df[columns_to_convert].dtypes)

order_approved_at                datetime64[ns]
order_delivered_customer_date    datetime64[ns]
shipping_limit_date              datetime64[ns]
order_purchase_timestamp         datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
review_answer_timestamp          datetime64[ns]
dtype: object
